# IBM Data Science Capstone: Where to Open a Coffee Shop?
## Author: Zack Kenyon

## Introduction

### Business Problem
This project will focus on determining the best location to open a coffee shop if you are wanting to take advantage of Arizona State University's very large student population. ASU has 4 campuses: Tempe, Downtown Phoenix, West (glendale) and Polytechnic (East Mesa). Each of these campuses offer vasitly different landscapes in regards to their surrounding areas. By analyzing these areas, the goal is to find neighborhoods near campus with an abundance of non-coffee related businesses. Consideration will also be given to the amount of major chains, Starbucks and Dutch Bros. compared to non-chains in the area.  

### Target Audience
The audience for a project such as this would be any aspiring restaurant owners, especially those interested in opening a cafe. Most restaurants close within a year of opening and while other factors certainly contribute to this, the hardest to overcome is a poor location as it is something that cannot easily be improved.    

## Data Collection 

### Geopy 
To convert the campus addresses to latitudes and longitudes, Geopy's nominatim package will be implemented.

### FourSquare API
Information regarding the surrounding businesses will be collected through use of FourSquare's API with each campus as the center point for searches. The radius will be set to 1600 as that gives approximately 1 miles worth of businesses to sample. From the results, the business names, categories, and locations will used to measure the denisty of coffee shops surrounding each campus, the number of chains, and finally number of other non-restuarant businesses. 

This information will ultimately determine which of campuses should be explored as the best option for opening a coffee shop as it gives insight into the amount of competition in the area and the amount of potential customers outside of just ASU. The best location being the one that minimizes the former while maximizing the latter. 

### Trending Venues
DISCLAIMER: Trending data will be excluded from this analysis due to the COVID-19 outbreak. As it stands, trending data will be largely skewed with many businesses, including ASU itself, being temporarily closed. 

In [23]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np 
import matplotlib as plt
import requests

from sklearn.cluster import KMeans 

!pip install -q geopy
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes
import folium

import json
with open('az_arizona_zip_codes_geo.min.json') as json_data: #this gives us the outline of zipcodes for choropleth later
    AZ_Zipcodes= json.load(json_data)   

Solving environment: ...working... done

# All requested packages already installed.



In [5]:
Campuses = ['NE_Tempe','SE_Tempe', 'Downtown Phx', 'West', 'Polytechnic']

NE_Tempe = '1001 S McAllister Ave. Tempe, Arizona'
SW_Tempe = '1151 S. Forest Ave. Tempe, Arizona'
DTP = '411 N. Central Ave. Phoenix, Arizona'
West = '13590 N. 47th Ave. Glendale, Arizona'
Poly = '7001 E Williams Field Rd. Mesa, Arizona'

addresses = [NE_Tempe, SW_Tempe, DTP, West, Poly]

Geolocator = Nominatim(user_agent = 'ZDK')
Lat = []
Long = []

for loc in addresses:
    location = Geolocator.geocode(loc)
    Lat.append(location.latitude)
    Long.append(location.longitude)

In [6]:
ASU = pd.DataFrame(list(zip(Campuses, addresses, Lat, Long)),  columns = {'Campus' : Campuses, 'Address' : addresses, 'Latitude' : Lat, 'Longitude': Long})

In [7]:
ASU.head()

,Campus,Address,Latitude,Longitude
0,NE_Tempe,"1001 S McAllister Ave. Tempe, Arizona",33.419357,-111.928768
1,SE_Tempe,"1151 S. Forest Ave. Tempe, Arizona",33.416972,-111.936246
2,Downtown Phx,"411 N. Central Ave. Phoenix, Arizona",33.452953,-112.073454
3,West,"13590 N. 47th Ave. Glendale, Arizona",33.560432,-112.160275
4,Polytechnic,"7001 E Williams Field Rd. Mesa, Arizona",33.307249,-111.683980


In [8]:
def getVenues(Campus, Lat, Long):
    Client_ID = 'HMK4WZSTRPQ03QBMQ2UXPF1O4M4DAW5K0HHWE4BW15OPI3D5'
    Client_Secret = 'XFPVKCL2QM42NJNHFK1A4BAY0BPZ1HTECLBG1JZHQ31BZ21F'
    Version = '20200320'
    LIMIT = 200 
    Radius = 1600 #collecting venues within roughly 1 mile from campus 
    
    Venues = []
    for Campus, Lat, Long in zip(Campus, Lat, Long):
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
            Client_ID, Client_Secret, Lat, Long, Version, Radius, LIMIT)
    
        results = requests.get(url).json()["response"]['groups'][0]['items']
        Venues.append([(
            Campus, 
            Lat, 
            Long, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for Venues in Venues for item in Venues])
    nearby_venues.columns = ['Campus', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
        

In [9]:
Campus_Venues = getVenues(ASU['Campus'], ASU['Latitude'], ASU['Longitude'])
Campus_Venues.head()

,Campus,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,NE_Tempe,33.419357,-111.928768,Dutch Bros. Coffee,33.416966,-111.926024,Coffee Shop
1,NE_Tempe,33.419357,-111.928768,Ahipoki Tempe,33.417570,-111.926048,Hawaiian Restaurant
2,NE_Tempe,33.419357,-111.928768,Raising Cane's Chicken Fingers,33.422364,-111.924376,Fried Chicken Joint
3,NE_Tempe,33.419357,-111.928768,Snooze an AM Eatery,33.423735,-111.934702,Breakfast Spot
4,NE_Tempe,33.419357,-111.928768,Original ChopShop Co.,33.422225,-111.935875,American Restaurant


In [10]:
#Building the list of zipcodes surrounding each campus by 
Zip = [] 
for lat, long in zip(Campus_Venues['Venue Latitude'], Campus_Venues['Venue Longitude']):
    Venue_Zip = Geolocator.reverse((lat, long))
    Zip.append(Venue_Zip.raw['address']['postcode']) #collecting only Zipcodes

#The GeoJson file only contains 5-digit codes, so any nine digit code needs to be trimmed to 5-digit form    
for i in range(0, len(Zip)): 
    if len(Zip[i]) > 5:
        Zip[i] = Zip[i][0:5]

In [11]:
Campus_Venues.insert(4, 'Zipcode', Zip)

#Recombining NE & SE Tempe and dropping duplicates that may have been picked up in their overlapping search radius 
Campus_Venues.drop_duplicates(['Venue Latitude', 'Venue Longitude'], keep = 'first', inplace =True)
Campus_Venues['Campus'].replace(['SE_Tempe', 'NE_Tempe'],['Tempe', 'Tempe'], inplace = True)

#getting a sense of the venues in each zipcode
Counted_Venues = Campus_Venues.drop(['Latitude', 'Longitude', 'Venue Category', 'Venue Latitude', 'Venue Longitude', 'Zipcode'], axis = 1).groupby('Campus').count()
Counted_Venues

,Venue
Campus,
Downtown Phx,100
Polytechnic,39
Tempe,120
West,71


### Number of Venues within 1 Mile of Each Campus
The Tempe campus was divided into 2 segments due to its size. Duplicates were then removed from the DF being sure to check only the subset of 'Venue Latitude' and 'Venue Longitude'. The 2 segments were then recombined to Tempe with 80 overlapping venues being removed.    

Regarding the number of venues, the first thing to notice here is that Polytechnic is the least saturated of the four campuses, followed by the west campus, then Downtown and finally Tempe. Looking at the documentation for Forsquare, 100 venues appears to be the max number that can be called with my current credentials. For this reason, the Downtown numbers may actually be appear than whats listed here. However, unlike tempe, the downtown campus is actually the smallest campus and thus, it wouldnt make sense to have 2 separate location coordinates to pull from.  

Moving forward, we will use the zipcodes surrounding the campuses to analyze the total venues in the area, total coffee shops, and number of chains.  

In [12]:
Hot_Campus = pd.get_dummies(Campus_Venues[['Venue Category']], prefix = '', prefix_sep = '')
Hot_Campus.insert(0, 'Zipcode', Campus_Venues['Zipcode'])
totals = Hot_Campus[['Zipcode', 'Coffee Shop']]
totals = totals.groupby('Zipcode').sum()
Hot_Campus = Hot_Campus.groupby('Zipcode').mean()
Hot_Campus

,Airport,Airport Service,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
Zipcode,,,,,,,,,,,,,,,,,,,,,
85003,0.00,0.00,0.00,0.076923,0.000000,0.019231,0.000000,0.000000,0.000000,0.000000,...,0.038462,0.038462,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
85004,0.00,0.00,0.00,0.021739,0.021739,0.065217,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
85007,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
85034,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
85051,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
85212,0.12,0.04,0.08,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.000000,0.000000,0.040000,0.000000
85236,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
85280,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
85281,0.00,0.00,0.00,0.034091,0.000000,0.000000,0.000000,0.011364,0.011364,0.000000,...,0.022727,0.011364,0.011364,0.011364,0.011364,0.00,0.000000,0.011364,0.000000,0.022727


In [13]:
#Now that we have the one-hot matrix, we can determine the average frequency of coffee-shops in each Zipcode
Coffee_freq = Hot_Campus[['Coffee Shop']].reset_index()
Total_Coffee_Shops = totals[['Coffee Shop']].reset_index()
Coffee_freq

,Zipcode,Coffee Shop
0,85003,0.076923
1,85004,0.152174
2,85007,0.000000
3,85034,0.000000
4,85051,0.000000
5,85212,0.080000
6,85236,0.000000
7,85280,0.333333
8,85281,0.045455
9,85287,0.103448


In [14]:
#Venue Category needed to be collected here so the search term could include all Restaurant Variants from FourSquare
Restaurants = Campus_Venues[['Zipcode','Venue Category']]
Rest = Restaurants[Restaurants['Venue Category'].str.contains('Restaurant|Place|Joint')]
total_Rest = Rest.groupby('Zipcode').count().reset_index()
total_Rest

,Zipcode,Venue Category
0,85003,18
1,85004,10
2,85007,1
3,85051,1
4,85212,5
5,85236,3
6,85281,45
7,85287,12
8,85295,3
9,85301,4


In [15]:
All_Venues = Campus_Venues[['Zipcode', 'Venue']]
Chains = All_Venues[All_Venues['Venue'].str.contains('Starbucks|Dutch')]
Total_Chains = Chains.groupby('Zipcode').count().reset_index()
Total_Chains

,Zipcode,Venue
0,85003,1
1,85004,1
2,85212,1
3,85280,1
4,85281,1
5,85287,1
6,85302,1


In [16]:
Zip_Info = pd.merge_ordered(Coffee_freq, total_Rest)
Zip_Info = Zip_Info.fillna(0)
Zip_Info.rename(columns = {'Venue Category':'Total Restaurants', 'Coffee Shop' : 'Coffee Shop Freq'}, inplace = True)
Zip_Info.insert(2, 'Total Coffee Shops', Total_Coffee_Shops['Coffee Shop'])
Zip_Info

,Zipcode,Coffee Shop Freq,Total Coffee Shops,Total Restaurants
0,85003,0.076923,4,18.0
1,85004,0.152174,7,10.0
2,85007,0.000000,0,1.0
3,85034,0.000000,0,0.0
4,85051,0.000000,0,1.0
5,85212,0.080000,2,5.0
6,85236,0.000000,0,3.0
7,85280,0.333333,1,0.0
8,85281,0.045455,4,45.0
9,85287,0.103448,3,12.0


### Coffee Shops, Restaurants in Zipcode
We can see in the Zip_Info DF the frequency of coffee shops compared to other venues in the zipcode, along with total coffee shops and total restaurants. 

From this we can see the although 85280 has the greatest frequency, there is only 1 coffee shop and 0 restaurants. So frequency alone can be misleading. However we see there are a couple of tiers in regards to totals. We can pretty safely divide the total restaurants into >20, 20-10, <5. Something similar can be seen with total coffee shops. 

Moving forward, we will pull the top 10 venues for each zipcode and use that in coordination with Zip_Info for K-Means clustering. 

In [17]:
columns = []

for ind in np.arange(10):
    columns.append('#{} Most Common Venue'.format(ind+1))

Zip_Top10 = pd.DataFrame(columns=columns)

for row in Zip_Info['Zipcode']:
    freqs = Hot_Campus.loc[row,:]
    freqs = freqs.T.sort_values(ascending = False)
    freqs = freqs[0:10]
    top10 = freqs.index.values[0:10]
    Zip_Top10.loc[row, :] = top10.T

Zip_Top10.reset_index(inplace = True)
Zip_Top10.drop(['index'], axis = 1, inplace = True)

Zip_Info = Zip_Info.join(Zip_Top10, rsuffix = '')
Zip_Info.head()

,Zipcode,Coffee Shop Freq,Total Coffee Shops,Total Restaurants,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,#6 Most Common Venue,#7 Most Common Venue,#8 Most Common Venue,#9 Most Common Venue,#10 Most Common Venue
0,85003,0.076923,4,18.0,Coffee Shop,American Restaurant,Music Venue,Hotel,Pizza Place,Thai Restaurant,Sandwich Place,Pub,Park,Salon / Barbershop
1,85004,0.152174,7,10.0,Coffee Shop,Hotel,Art Gallery,Breakfast Spot,Cocktail Bar,Bar,New American Restaurant,Opera House,Pizza Place,Comedy Club
2,85007,0.000000,0,1.0,Pizza Place,Yoga Studio,Ethiopian Restaurant,Coffee Shop,College Baseball Diamond,College Stadium,Comedy Club,Comic Shop,Construction & Landscaping,Convenience Store
3,85034,0.000000,0,0.0,Museum,Yoga Studio,Cocktail Bar,College Baseball Diamond,College Stadium,Comedy Club,Comic Shop,Construction & Landscaping,Convenience Store,Deli / Bodega
4,85051,0.000000,0,1.0,Italian Restaurant,Grocery Store,Park,Hunting Supply,Smoke Shop,Convenience Store,Yoga Studio,Dive Bar,Discount Store,Dessert Shop


In [24]:
#Using folium map package to create a map of Arizona with a zoom_start that encompasses all 4 campuses
ASU_map = folium.Map(location = [Lat[0], Long[0]], zoom_start = 11)

#Creating a Choropleth using the Zipcode Data, and total number of Coffee Shops
ASU_map.choropleth(
    geo_data=AZ_Zipcodes,
    data=Zip_Info,
    columns=['Zipcode', 'Total Coffee Shops'],
    key_on='feature.properties.ZCTA5CE10',
    threshold_scale = [0, 1, 2, 4, 6, 8],
    fill_color='Blues', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Coffee Shops in Zipcode'
)

#Need to add markers for each of the campuses that outline the search radius
for campus, lat, long in zip(ASU['Campus'], ASU['Latitude'], ASU['Longitude']):
    folium.Circle(
        [lat, long], 
        radius = 1600, 
        fill = False).add_to(ASU_map)

for campus, lat, long in zip(ASU['Campus'], ASU['Latitude'], ASU['Longitude']):
    label = folium.Popup(str(campus))
    folium.Marker([lat, long], popup = label).add_to(ASU_map)
    
# display map
#ASU_map

# CHOROPLETH MAP HIDDEN TO DECREASE FILE SIZE BUT I HAVE IT IN MY REPORT AND PRESENTATION

### Coffee Shops Map
In the map above, a choropleth map was created using the geocode data for Arizona Zipcodes. Each campus was given a popup marker and circle to indicate the FourSquare search radius. Tempe kept the dual markers since the search radius was two separate rings. We can see from the graph that the highest number of coffee shops within 1 mile of an ASU campus are located on the East side of the downtown campus. The tempe campus and West side of Downtown have similar amounts of coffee shops. Finally, the West and Polytech campuses each only have 1-2 coffee shops in the surrounding zipcodes.   

In [19]:
#K-Means clustering based on a one-hot-encoded DF with freq, total shops/restaurants, and most common venues
clusters = 5
Zip_Cls = Zip_Info.drop(['Zipcode'], axis = 1)
Hot_Zip = pd.get_dummies(Zip_Cls)
K_Zip = KMeans(n_clusters = clusters, random_state = 0).fit(Hot_Zip)
K_Zip.labels_ 

array([3, 0, 1, 1, 1, 4, 4, 1, 2, 0, 4, 4, 3])

In [20]:
#Recombing libraries
Clustered_Zip = Zip_Info.copy(deep = True)
Clustered_Zip.insert(1, 'Cluster', K_Zip.labels_)
Clustered_Zip.sort_values('Cluster')

,Zipcode,Cluster,Coffee Shop Freq,Total Coffee Shops,Total Restaurants,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,#6 Most Common Venue,#7 Most Common Venue,#8 Most Common Venue,#9 Most Common Venue,#10 Most Common Venue
1,85004,0,0.152174,7,10.0,Coffee Shop,Hotel,Art Gallery,Breakfast Spot,Cocktail Bar,Bar,New American Restaurant,Opera House,Pizza Place,Comedy Club
9,85287,0,0.103448,3,12.0,Coffee Shop,Deli / Bodega,Mexican Restaurant,Athletics & Sports,Public Art,Dive Bar,Burger Joint,Sandwich Place,College Baseball Diamond,Middle Eastern Restaurant
2,85007,1,0.000000,0,1.0,Pizza Place,Yoga Studio,Ethiopian Restaurant,Coffee Shop,College Baseball Diamond,College Stadium,Comedy Club,Comic Shop,Construction & Landscaping,Convenience Store
3,85034,1,0.000000,0,0.0,Museum,Yoga Studio,Cocktail Bar,College Baseball Diamond,College Stadium,Comedy Club,Comic Shop,Construction & Landscaping,Convenience Store,Deli / Bodega
4,85051,1,0.000000,0,1.0,Italian Restaurant,Grocery Store,Park,Hunting Supply,Smoke Shop,Convenience Store,Yoga Studio,Dive Bar,Discount Store,Dessert Shop
7,85280,1,0.333333,1,0.0,Coffee Shop,Pool Hall,Convenience Store,Yoga Studio,Ethiopian Restaurant,College Baseball Diamond,College Stadium,Comedy Club,Comic Shop,Construction & Landscaping
8,85281,2,0.045455,4,45.0,Pizza Place,Bar,Sandwich Place,Coffee Shop,American Restaurant,Chinese Restaurant,Restaurant,Breakfast Spot,Brewery,Burger Joint
0,85003,3,0.076923,4,18.0,Coffee Shop,American Restaurant,Music Venue,Hotel,Pizza Place,Thai Restaurant,Sandwich Place,Pub,Park,Salon / Barbershop
12,85302,3,0.036364,2,22.0,Mexican Restaurant,Pizza Place,Grocery Store,Chinese Restaurant,Donut Shop,Fast Food Restaurant,Intersection,Pharmacy,Coffee Shop,Smoke Shop
5,85212,4,0.080000,2,5.0,Rental Car Location,Airport,Hotel,Coffee Shop,Airport Terminal,Video Store,Fried Chicken Joint,Convenience Store,Business Service,Brewery


In [21]:
#To get FULL_DATA, merge Campus Venues w/ Clustered_Zip on the Zipcode column. The outer merge and m:m validation 
#ensure that the DFs with different shapes can be merged on their 1 similar collumn. 
Full_Data = pd.merge(Campus_Venues, Clustered_Zip, on = 'Zipcode', how = 'outer', validate = 'many_to_many')
Full_Data.head()

,Campus,Latitude,Longitude,Venue,Zipcode,Venue Latitude,Venue Longitude,Venue Category,Cluster,Coffee Shop Freq,...,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,#6 Most Common Venue,#7 Most Common Venue,#8 Most Common Venue,#9 Most Common Venue,#10 Most Common Venue
0,Tempe,33.419357,-111.928768,Dutch Bros. Coffee,85281,33.416966,-111.926024,Coffee Shop,2,0.045455,...,Pizza Place,Bar,Sandwich Place,Coffee Shop,American Restaurant,Chinese Restaurant,Restaurant,Breakfast Spot,Brewery,Burger Joint
1,Tempe,33.419357,-111.928768,Snooze an AM Eatery,85281,33.423735,-111.934702,Breakfast Spot,2,0.045455,...,Pizza Place,Bar,Sandwich Place,Coffee Shop,American Restaurant,Chinese Restaurant,Restaurant,Breakfast Spot,Brewery,Burger Joint
2,Tempe,33.419357,-111.928768,Original ChopShop Co.,85281,33.422225,-111.935875,American Restaurant,2,0.045455,...,Pizza Place,Bar,Sandwich Place,Coffee Shop,American Restaurant,Chinese Restaurant,Restaurant,Breakfast Spot,Brewery,Burger Joint
3,Tempe,33.419357,-111.928768,Devil's Advocate Bar and Grill,85281,33.421791,-111.924444,Sports Bar,2,0.045455,...,Pizza Place,Bar,Sandwich Place,Coffee Shop,American Restaurant,Chinese Restaurant,Restaurant,Breakfast Spot,Brewery,Burger Joint
4,Tempe,33.419357,-111.928768,Postino Annex,85281,33.423973,-111.934752,Wine Bar,2,0.045455,...,Pizza Place,Bar,Sandwich Place,Coffee Shop,American Restaurant,Chinese Restaurant,Restaurant,Breakfast Spot,Brewery,Burger Joint


In [22]:
#Creating the base layer for the map
Full_map = folium.Map([Full_Data['Latitude'].iloc[0], Full_Data['Longitude'].iloc[0]], zoom_start = 11)

#A list of colors to assig the clusters 
Colors = ['lightblue', 'darkgreen', 'yellow', 'orange', 'purple']

#Creating the Feature groups so user can toggle the clusters 
fg0 = folium.FeatureGroup(name = 'Cluster: 0, Light Blue')
Full_map.add_child(fg0)
fg1 = folium.FeatureGroup(name = 'Cluster: 1, Dark Green')
Full_map.add_child(fg1)
fg2 = folium.FeatureGroup(name = 'Cluster: 2, Yellow')
Full_map.add_child(fg2)
fg3 = folium.FeatureGroup(name = 'Cluster: 3, Orange')
Full_map.add_child(fg3)
fg4 = folium.FeatureGroup(name = 'Cluster: 4, Purple')
Full_map.add_child(fg4)

#Creating circle markers for every venue collected in the intial search and assigning them a color and featuregroup
#based on its assigned cluster 
for cluster, lat, long in zip(Full_Data['Cluster'], Full_Data['Venue Latitude'], Full_Data['Venue Longitude']):
    if cluster == 0:
        folium.CircleMarker(
            [lat, long],
            radius = 7,
            popup = folium.Popup('Cluster: ' + str(cluster)),
            weight = 0.1,
            fill = True,
            fill_color = Colors[int(cluster)],
            fill_opacity = 1
            ).add_to(fg0)
    elif cluster == 1:
        folium.CircleMarker(
            [lat, long],
            radius = 7,
            popup = folium.Popup('Cluster: ' + str(cluster)),
            weight = 0.1,
            fill = True,
            fill_color = Colors[int(cluster)],
            fill_opacity = 1
            ).add_to(fg1)
    elif cluster == 2:
        folium.CircleMarker(
            [lat, long],
            radius = 7,
            popup = folium.Popup('Cluster: ' + str(cluster)),
            weight = 0.1,
            fill = True,
            fill_color = Colors[int(cluster)],
            fill_opacity = 1
            ).add_to(fg2)
    elif cluster == 3:
        folium.CircleMarker(
            [lat, long],
            radius = 7,
            popup = folium.Popup('Cluster: ' + str(cluster)),
            weight = 0.1,
            fill = True,
            fill_color = Colors[int(cluster)],
            fill_opacity = 1
            ).add_to(fg3)
    elif cluster == 4:
        folium.CircleMarker(
            [lat, long],
            radius = 7,
            popup = folium.Popup('Cluster: ' + str(cluster)),
            weight = 0.1,
            fill = True,
            fill_color = Colors[int(cluster)],
            fill_opacity = 1
            ).add_to(fg4) 

#Showing the search radius and campus location for all 4 campuses
for campus, lat, long in zip(ASU['Campus'], ASU['Latitude'], ASU['Longitude']):
    folium.Circle(
        [lat, long], 
        radius = 1600, 
        fill = False).add_to(Full_map)

for campus, lat, long in zip(ASU['Campus'], ASU['Latitude'], ASU['Longitude']):
    label = folium.Popup(str(campus))
    folium.CircleMarker(
        [lat, long], 
        radius= 10, 
        fill = True, 
        fill_opacity = .7, 
        fill_color = 'darkred', 
        popup = label).add_to(Full_map)   
    
folium.LayerControl(collapsed = False).add_to(Full_map)   #collapsed set to false so users can see additional functionality
Full_map

## Cluster Map
There are a total of 5 clusters which can be toggled on and off to get a view of the general areas around each campus. Due to the limitations of foliums's choropleth map, the (Lat, Long) data for each venue was utilized to create CircleMarkers, color coordinated by cluster, which would then 'paint' the area where that cluster is prominent. This method helps to visualize the various groupings at glance which can then be further explored.

**When restaring the kernel, the cluster contents stay the same but the labels for clusters 0 and 3 flip almost everytime. For some reason, these two are the only ones to change labels. For clarification, just pay attention to the two locations the clusters show up at.**   

1)
**Cluster 0/Cluster 3** was the most common in terms of venues but only consisted of 2 Zipcodes located primarily on the southwest side of the Downtown campus and on the north side of the west campus. The 2 Zipcodes contained in Cluster 0 contain 18 and 24 total restuarants. As mentioned previously, there appears to be a couple obvious tiers in that regard. The top10 venues only have two categories in common: Pizza Place (1st and 3rd) and coffee shop (2nd and 9th). 

2)
**Cluster 1** was the most common cluster with 5 Zipcodes but it was least populated with venues at with only 13. Each Zipcode in Cluster 1 has 0-1 coffee shops and less than 5 restaurants within them. With 5 Zipcodes in this cluster, it does appear at all four campuses. Its top10 venues are nearly identical amongst 3 of the 5 zipcodes with major venues being museums, yoga studios and college stadiums. The remaining two are likewise very similar amongst its 5-10 venues, sharing commonality with the rest of the group in the presence of yoga studios. 

A safe assumption would be that the similarity between these zipcodes is misleading. After all venues were listed, the most common remaining frequncy would have been 0 which could explain why the latter half of top10 lists match so well.  

3)
**Cluster 2** was the second most populated cluster with 88 venues packed into only 1 zipcode located at the Tempe campus. It has 4 coffee shops and 45 restaurants giving it more restaurants than clusters 1, 3 and 4 combined. The majority of Cluster 2 is located on the northwest side of the Tempe campus making that location arguably the most competitive for opening a business. 

4)
**Cluster 3/Cluster 0** contains 75 venues spread across 2 zipcodes. The majority of cluster 2 is located on the east side of the downtown campus. Both Zipcodes of cluster three have a coffee shop as their most common venue with breakfast and deli shops following close behind. A coffee shop in cluster 3 would yield the most direct competition. 

5)
**Cluster 4** is similar to cluster 1 in that it has very few coffee shops and restaurants spread out amongst its 3 zipcodes. It is located primarily at the Polytechnic campus, a few at West, having 46 venues with only 2 of them are coffee shops and 12 are restaurants. Further, the top 10 venues have next to no commonality making a decision regarding viability on this cluster a bit difficult.